In [1]:
from pathlib import Path
import os
import sys
import gc
import shutil
import json
import math
import uuid
from collections import defaultdict
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, NamedTuple, Callable, Iterable, Set, Optional, Any
import scml
from mylib.preprocess.en import BasicPreprocessor, labels

In [2]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB
device=1, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [3]:
#target_size = 100

In [4]:
df = pd.read_parquet("input/part00.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            100 non-null    object
 1   prompt          100 non-null    object
 2   street_address  100 non-null    object
 3   student_name    100 non-null    object
 4   username        100 non-null    object
 5   personal_url    100 non-null    object
 6   model           100 non-null    object
dtypes: object(7)
memory usage: 5.6+ KB


In [5]:
df.sample(5).head(5)

,text,prompt,street_address,student_name,username,personal_url,model
17,"\n \nDesign thinking is a problem-solving approach that fosters innovation and creativity in maritime engineering and naval architecture. This process encourages engineers to think outside the box and develop unique solutions to complex challenges. The design thinking process involves several stages, including brainstorming, prototyping, and research.\n\nOne example of design thinking in action occurred at an office frequented by Amber Samra, located near 71 Justin squares South Deantown WR2E 2HP. Engineers were tasked with designing a more efficient cargo ship for transporting goods across the ocean. Using the design thinking process, they began by conducting extensive research into existing ships and their weaknesses. Through this research, they discovered that many cargo ships had inefficient hull shapes that created drag and wasted fuel.\n\nTo address this issue, the team moved on to the brainstorming stage of the design thinking process. During this stage, they generated numerous ideas for improving the hull shape and reducing drag. One engineer suggested incorporating air bubble technology to reduce friction between the ship and the water. Another proposed using a computer algorithm to optimize the hull shape based on hydrodynamic principles.\n\nAfter selecting the best ideas from their brainstorming session, the team moved on to prototyping. They constructed a small-scale model of the ship with the new hull shape and conducted experiments in a wave tank. These experiments allowed them to test the effectiveness of their design and make any necessary adjustments before building a full-scale prototype.\n\nThe final step in the design thinking process was to evaluate the success of the project. The team found that their new hull shape significantly reduced drag and improved fuel efficiency, resulting in substantial cost savings for shipping companies.\n\nAnother example of design thinking in action involved a marine engineer named ___aarush__xxx__thomas__official__ who worked on a project related to Facebook.com/--shannon-4931-friis-official-. The challenge was to create a more eco-friendly way for cruise ships to generate electricity while at sea.\n\nUsing the design thinking process, ___aarush__xxx__thomas__official__ first conducted extensive research into alternative energy sources and the specific needs of cruise ships. He then moved on to brainstorming, where he generated ideas such as installing wind turbines or using solar panels. However, these ideas proved impractical due to space constraints and weather conditions.\n\nIn the end, ___aarush__xxx__thomas__official__ decided to explore the possibility of using wave energy converters (WECs). WECs convert the motion of ocean waves into electricity, making them an ideal solution for generating power at sea. After creating a prototype and testing it extensively, ___aarush__xxx__thomas__official__ successfully implemented the WEC system on a cruise ship, providing a renewable source of energy and reducing carbon emissions.\n\nThe design thinking process differs from traditional problem-solving methods by focusing on empathy, ideation, and experimentation rather than relying solely on analytical techniques. By incorporating brainstorming, prototyping, and research into each project, designers are able to approach problems in a more holistic way, leading to innovative and creative solutions. The examples above demonstrate how the design thinking process can be applied in maritime engineering and naval architecture to tackle complex challenges and promote sustainability.","[INST] Write a short essay on the following topic:\n In Maritime Engineering and Naval Architecture, how does the design thinking process foster innovation and creativity within various activities? Explore the role of brainstorming, prototyping, and research in developing innovative ideas and approaches. Discuss how the design thinki

In [6]:
pre = BasicPreprocessor()
rows = []
for t in tqdm(df.itertuples()):
    street_address = str(getattr(t, "street_address")),
    student_name = str(getattr(t, "student_name")),
    username = str(getattr(t, "username")),
    personal_url = str(getattr(t, "personal_url")),
    text = pre(getattr(t, "text"))
    tokens, trailing_whitespace = pre.tokenize(text)
    rows.append({
        "document": f"rh_{uuid.uuid4().hex}",
        "full_text": text,
        "tokens": tokens,
        "trailing_whitespace": trailing_whitespace,
        "labels": labels(text=text, tokens=tokens, ),
        "street_address": street_address,
        "student_name": student_name,
        "username": username,
        "personal_url": personal_url,
    })


100it [00:00, 399.34it/s]


In [7]:
%%time
with open("output/rh01.json", "w") as f:
    json.dump(rows, f)

CPU times: user 15 ms, sys: 4.06 ms, total: 19.1 ms
Wall time: 31.2 ms


In [8]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:00:00.530536
